<a href="https://colab.research.google.com/github/McPatrik/Data-analyst---Projects./blob/main/A_B_test_shop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача

Продуктовый онлайн магазин, хочет провести исследование на тему, каким образом стоимость доставки влияет на количество и сумму заказов. 2 тестовые группы, у одной по умолчанию стоимость доставки 500 рублей, у другой изменение, а именно - 200 рублей. Количество пользователей доставкой(регулярных) на данный момент 200 000, время эксперимента 30 дней.

Провести анализ результатов A/B тестирования с использованием инструментов математической статистики для интерпретации результатов и принятия решения касательно стоимости доставки.

**Оцениваемые метрики**
- Средняя выручка на одного пользователя (ARPU)
- Среднее количество посещений
- Конверсия (добавление в корзину/покупка)

# Полученные данные

В интернете не было найдено данных для сложных A/B тестов, по этой причине данные были сгенерированы.
\
По результатам тестирования 5 файлов:
- добавление товаров в корзину пользователем, содержащий в себе id товара, id пользователя, и дата добавления товара в корзину
- посещения приложения пользователем, содержащий в себе дату посещения и id пользователя
- покупка товаров, содержащий в себе id пользователя, id товара, и дату покупки
- файл со стоимостью каждого товара, id товара и его стоимость
- информация о том к какой тестовой группе принадлежит пользователь, id пользователя, группа (A,B)

In [344]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

In [345]:
# чтение файлов
to_cart = pd.read_csv('/content/drive/MyDrive/AB_to_cart.csv')
visits = pd.read_csv('/content/drive/MyDrive/AB_visits_.csv')
all_items = pd.read_csv('/content/drive/MyDrive/AB_all_items.csv')
users = pd.read_csv('/content/drive/MyDrive/AB_users.csv')
orders = pd.read_csv('/content/drive/MyDrive/AB_orders.csv')

In [67]:
# all_items = all_items.groupby('item_id', as_index=False).mean()
# all_items.to_csv('/content/drive/MyDrive/AB_all_items.csv', index=False)

## Предварительная обработка данных

In [346]:
#удаление дубликатов пользователей
users.drop_duplicates(inplace=True)

In [347]:
# просмотр количества пользователей в тестовой и контрольной группе.
users.groupby('group').count()

,user_id
group,
A,95152
B,95135


In [348]:
all_items.describe()

,item_id,item_cost
count,7.780090e+05,778009.000000
mean,1.049708e+06,5689.670674
std,5.483765e+05,13304.455870
min,1.000040e+05,1.000000
25%,5.743260e+05,1262.590000
50%,1.050448e+06,2039.400000
75%,1.524017e+06,3363.480000
max,1.999998e+06,139091.000000


# Метрики.

- Среднее количество посещений
- Визуализация таких метрик как DAU и WAU
- Средняя выручка на одного пользователя (ARPU)
- Конверсия, добавленных товаров в корзину/купленных товаров

**Среднее количество посещений в группах.**


Преобразуем данные с проставлением 0, где пользователь существует, но не посещал наше приложение. Для того чтобы в дальнейшем оценить количество пользователей посетивших приложение со всеми пользователями приложения.

In [349]:
users_visits = users.merge(visits, on='user_id', how='left')
users_visits = users_visits.fillna(0)
users_visits

,user_id,group,date
0,1651144,A,2023-06-04
1,1651144,A,2023-06-05
2,1651144,A,2023-06-07
3,1651144,A,2023-06-09
4,1651144,A,2023-06-13
...,...,...,...
2095353,2419769,B,2023-06-10
2095354,2419769,B,2023-06-19
2095355,2419769,B,2023-06-21
2095356,2419769,B,2023-06-22


In [350]:
users_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095358 entries, 0 to 2095357
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  int64 
 1   group    object
 2   date     object
dtypes: int64(1), object(2)
memory usage: 63.9+ MB


In [78]:
# users_visits[(users_visits.group == 'A')&
#              (users_visits.date >= '2023-06-22')&
#               (~users_visits.user_id.isin(to_cart.user_id.unique()))].sample(frac=0.5)

Находим количество посещений каждого пользователем за время теста, учитывая тех кто не посещал приложение, но находиться в нашей базе.

In [351]:
count_visits = users_visits[users_visits['date'] != 0].groupby(['user_id','group']).count().reset_index()
count_visits = pd.concat([count_visits, users_visits[users_visits['date'] == 0]])
count_visits

,user_id,group,date
0,1000008,A,10
1,1000025,A,11
2,1000034,A,15
3,1000036,A,11
4,1000042,A,11
...,...,...,...
1925729,2677604,B,0
1926482,2243622,B,0
1987978,2641121,B,0
2022840,2322583,B,0


Нахождение среднего количества посещений в 2-х группах.

In [352]:
mean_visits_group = count_visits.groupby(['group'])['date'].mean().reset_index()
mean_visits_group

,group,date
0,A,10.745176
1,B,11.277669


In [353]:
# сохранение выводов результатов посещения пользователей, среднее кол-во, p-value
visits_result = [list(mean_visits_group['group'].values), \
                 list(mean_visits_group['date'].values),\
                 stats.ttest_ind(list(count_visits[count_visits['group'] == 'A']['date']),list(count_visits[count_visits['group'] == 'B']['date'])).pvalue]
visits_result

[['A', 'B'], [10.745176139229864, 11.277668576233774], 4.971252950526728e-58]

**Графики DAU и WAU**

Для того чтобы ответить на вопрос каким образом изменился трафик в магазине, построим графики пользовательской активности за день и неделю - *DAU, WAU.*

In [354]:
visits_ = visits.merge(users, on='user_id', how='left')
df_dau = visits_.groupby(['date', 'group'])['user_id'].nunique()
df_dau = df_dau.reset_index()

In [355]:
fig = px.line(df_dau, x="date", y="user_id", color="group", title='DAU')
fig.show()

На графике DAU, наша тестовая группа показывает больший рост по сравнению с контрольной(до 21 июня), затем происходит снижение (необходимо изучить причины снижения, так как есть причина резкого спада, нет возможности так как данные синтетические). Контрольная группа, начиная с 20 июня показывает рост, к окончанию теста обе группы показывают снижение. Сложно сделать окончательные и достоверные выводы исходя из графика.

In [356]:
# преобразование в формат "даты"
visits_['date'] = pd.to_datetime(visits_['date'])

In [357]:
# функция для подсчёта метрики WAU
def wau(df):
  df_ = df.copy()
  df_date_max = df_['date'].max()
  x = df_['date'][0]
  y = x + timedelta(days=6)
  result_A = []
  result_B = []
  date = []
  while(y <= df_date_max):
    result_A.append(df_[(df_['date'] >= x) & (df_['date'] <= y)].groupby(['group'])['user_id'].nunique().values[0])
    result_B.append(df_[(df_['date'] >= x) & (df_['date'] <= y)].groupby(['group'])['user_id'].nunique().values[1])
    date.append(y)
    x = x + timedelta(days=1)
    y = y + timedelta(days=1)
  return result_A,result_B,date

In [358]:
r_wau = wau(visits_)
data_wau = {'date': r_wau[2], 'group A': r_wau[0], 'group B': r_wau[1]}
df_wau = pd.DataFrame(data_wau)

In [359]:
fig = px.line(df_wau, x="date", y=["group A", "group B"], title='WAU')
fig.show()

На графике WAU также есть тенденция роста, но более плавная, как и падение к концу месяца. Также мы видим пересечение по количеству трафика 27 июня.

**Нахождение средней выручки на пользователя**

In [360]:
#ARPU
#добавление стоимость каждого из товаров
orders_cost = orders.merge(all_items, on='item_id')
#маркировка пользователя группой
orders_cost = orders_cost.merge(users, on='user_id', how='right')
#заполнение пропусков значением 0
orders_cost = orders_cost.fillna(0)

In [361]:
sum_on_user = orders_cost.groupby(['user_id', 'group'])['item_cost'].sum().reset_index()
arpu = sum_on_user.groupby('group')['item_cost'].mean().reset_index()
arpu

,group,item_cost
0,A,15115.314137
1,B,17029.498763


In [362]:
# сохранение выводов результатов, arpu, p-value
arpu_result = [list(arpu['group'].values), \
                 list(arpu['item_cost'].values),\
                 stats.ttest_ind(list(sum_on_user[sum_on_user['group'] == 'A']['item_cost']),list(sum_on_user[sum_on_user['group'] == 'B']['item_cost'])).pvalue]
arpu_result

[['A', 'B'], [15115.314136873985, 17029.49876253094], 2.3171372380012462e-54]

**Количество добавленных товаров в корзину каждым пользователем.**

In [363]:
# объединение пользователей с их группой(информация по добавлениям в корзину)
to_cart_ = to_cart.merge(users, on='user_id', how='right')
to_cart_ = to_cart_.fillna(0)
# количество добавлений товаров в корзину каждого пользователя
to_cart_users = to_cart_[to_cart_['item_id'] != 0].groupby(['user_id', 'group'])['item_id'].count().reset_index()
to_cart_users = pd.concat([to_cart_users, to_cart_[to_cart_['item_id'] == 0][['user_id', 'item_id', 'group']]])
to_cart_users

,user_id,group,item_id
0,1000008,A,6.0
1,1000025,A,4.0
2,1000034,A,6.0
3,1000036,A,9.0
4,1000044,A,9.0
...,...,...,...
1047164,2773744,B,0.0
1047165,2253578,B,0.0
1047212,2287387,B,0.0
1047213,2560797,B,0.0


In [364]:
to_cart_gr = to_cart_[to_cart_['item_id'] != 0].groupby(['date', 'group'])['item_id'].count().reset_index()
fig = px.line(to_cart_gr, x="date", y='item_id', color='group', title='To_Cart')
fig.show()

На графике видно, что по добавлениям в корзину очевидно лидирует группа B, также группа A до 24 июня не показывает больших колебаний. Начиная с 22 июня, происходит пересечение показателей 2-х групп. С 28 июня обе группы показывают плавный спад показателей.

**Количество купленных товаров каждым пользователем**

In [365]:
# Количество купленных товаров каждым пользователем
number_orders = orders_cost[orders_cost['item_id'] != 0].groupby(['user_id', 'group'])['item_id'].count().reset_index()
number_orders = pd.concat([number_orders, orders_cost[orders_cost['item_id'] == 0][['user_id', 'item_id', 'group']]])
number_orders.rename(columns={'item_id': 'to_order'}, inplace=True)
number_orders

,user_id,group,to_order
0,1000008,A,3.0
1,1000025,A,2.0
2,1000034,A,4.0
3,1000036,A,4.0
4,1000044,A,5.0
...,...,...,...
679071,2253578,B,0.0
679100,2287387,B,0.0
679101,2560797,B,0.0
679102,2637344,B,0.0


In [366]:
fig = px.line(orders_cost[orders_cost['item_id'] != 0].groupby(['date', 'group'])['item_id'].count().reset_index(), x="date", y='item_id', color='group', title='To_Order')
fig.show()

Тестовая группа также лидирует по количеству купленных товаров.

**Конверсия, добавленных товаров в корзину/купленных товаров**

In [367]:
# cлияние добавленных товаров и купленных товаров для линеаризации метрик и подсчёта конверсии
number_orders = number_orders.merge(to_cart_users, on=['user_id', 'group'])
number_orders.rename(columns={'item_id': 'to_cart'}, inplace=True)
number_orders

,user_id,group,to_order,to_cart
0,1000008,A,3.0,6.0
1,1000025,A,2.0,4.0
2,1000034,A,4.0,6.0
3,1000036,A,4.0,9.0
4,1000044,A,5.0,9.0
...,...,...,...,...
190282,2253578,B,0.0,0.0
190283,2287387,B,0.0,0.0
190284,2560797,B,0.0,0.0
190285,2637344,B,0.0,0.0


In [368]:
# конверсия
conv = number_orders.groupby('group')['to_order'].sum().reset_index()
conv['to_cart'] = number_orders.groupby('group')['to_cart'].sum().values
conv['conv'] = conv['to_order']/conv['to_cart']
conv

,group,to_order,to_cart,conv
0,A,263788.0,466957.0,0.564909
1,B,357585.0,533807.0,0.669877


Как мы видим показатель конверсии в тестовой группе выше. Тестовая группа не только больше добавляет товаров в корзину, но и имеет выше показатель выкупа по сравнению с контрольной.

Посчёт линеаризации для перехода в новое признаковое пространство: к пропорциональной, но более чувствительной метрике.

In [369]:
number_orders['lr_cr'] = number_orders['to_order'] - (number_orders['to_cart']*conv[conv['group'] == 'A']['conv'].values)


In [370]:
conversion = [list(to_cart_users['group'].unique()),\
              conv['conv'].values,\
             stats.ttest_ind(number_orders[number_orders['group'] == 'A']['lr_cr'],number_orders[number_orders['group'] == 'B']['lr_cr'])[1]]
conversion

[['A', 'B'], array([0.56490855, 0.66987694]), 0.0]

In [371]:
result_df = {'Metric\'s A vs B': ['Avg_visits', 'ARPU', 'Conversion'],
            'Group_A':[visits_result[1][0], arpu_result[1][0], conversion[1][0]],
            'Group_B':[visits_result[1][1], arpu_result[1][1], conversion[1][1]] ,
            'Pvalue':[visits_result[2], arpu_result[2], conversion[2]]}
df = pd.DataFrame(result_df)
df

,Metric's A vs B,Group_A,Group_B,Pvalue
0,Avg_visits,10.745176,11.277669,4.971253e-58
1,ARPU,15115.314137,17029.498763,2.317137e-54
2,Conversion,0.564909,0.669877,0.000000e+00


 Подсчёт разницы между контрольной и тестовой группой.

 Установка уровня значимости равным 0.05, для сравнения с полученным p-value

In [372]:
df['Difference %'] = ((df['Group_A'] - df['Group_B'])/df['Group_A'])*100
df['Interpretation'] = df['Pvalue'] < 0.05

Использование поправки на множественную проверку гипотез для устранения эффекта множественных сравнений. Мной был сделан выбор использовать поправку метода Холма.

In [373]:
Amendment_Holm_Bonferroni = multipletests(df['Pvalue'], is_sorted=False, method='holm')
df['Amendment Holm–Bonferroni'] = Amendment_Holm_Bonferroni[0]

In [374]:
# результативная таблица.
df

,Metric's A vs B,Group_A,Group_B,Pvalue,Difference %,Interpretation,Amendment Holm–Bonferroni
0,Avg_visits,10.745176,11.277669,4.971253e-58,-4.955642,True,True
1,ARPU,15115.314137,17029.498763,2.317137e-54,-12.663876,True,True
2,Conversion,0.564909,0.669877,0.000000e+00,-18.581485,True,True


# **Вывод**
Гипотеза: снижение стоимости доставки оказывает положительное влияние на продуктовые метрики.

По результатам эксперимента получены следующий прирост в тестовой относительно контроля:
- Среднее количество посещений выше на 4.9%
- Средняя выручка на пользователя выше на 12.6%
- Показатель конверсии (добавление в корзину/покупка) выше на 18.5%


С 21- го числа наблюдается схождение метрик между группами.
Это может свидетельствовать о:
- наличии влияния сторонних факторов на эксперимент, необходимо проверить(но данные синтетические)
- наличии долгосрочного эффекта на метрики, который не удалось заметить в рамках теста.